# ***EJERCICIO 6***

In [1]:
import numpy as np
import pandas as pd

Se definen los vectores que guardan los valores de las funciones de densidad

In [57]:
# FUNCIONES DE DENSIDAD
f_x = np.array([0.135335283236613,0.270670566473225,0.270670566473225,0.180447044315484,
                0.090223522157742,0.036089408863097,0.012029802954366,0.003437086558390,
                0.000859271639598,0.000190949253244,0.000046498075017])

f_y = np.array([0.135335283236613,0.270670566473225,0.270670566473225,0.180447044315484,
                0.090223522157742,0.036089408863097,0.012029802954366,0.003437086558390,
                0.000859271639598,0.000190949253244,0.000046498075017])

**NOTA: Aquí en Python, la suma de los datos de las densidades no es igual a uno, difiere por un decimal muy pequeño y eso afecta en los cálculos.**

In [62]:
print(sum(f_x))
print(sum(f_x)==1)

1.0000000000000009
False


Después se crea la función que tiene como argumentos las densidades y sus rangos. Lo que hace es ir multiplicando los valores con la técnica vista en clase (va aumentando en uno y disminuye en el otro), solo cambia en algunos casos dependiendo desde donde se empiea a multiplicar y hasta donde termina el proceso (depende del tamaño de la densidad más pequeña).

In [43]:
def convolusion(v1, v2, rx, ry):
    # Ver cual de las dos densidades tiene menor tamaño
    if len(v1) <= len(v2):
        fx = v1
        fy = v2
    else:
        fx = v2
        fy = v1

    # Definir el Rango de la Suma de X + Y
    rango_suma = np.arange(min(rx) + min(ry), max(rx) + max(ry) + 1)

    densidad = []  # Lista donde se irán guardando los resultados

    # Se va a llenar desde 1 hasta el número de valores en el rango
    for k in range(len(rango_suma)):
        if k == 0:
            valor = fx[k] * fy[k]
            densidad.append(valor)
        elif k < len(fx):
            valor = 0
            for i in range(k + 1):
                valor += fx[i] * fy[k - i]
            densidad.append(valor)
        elif len(fx) != len(fy) and k < len(fy):
            valor = 0
            for a in range(len(fx)):
                valor += fx[a] * fy[k - a]
            densidad.append(valor)
        else:
            valor = 0
            for b in range(k - max(ry), len(fx)):
                valor += fx[b] * fy[k - b]
            densidad.append(valor)

    return np.array(densidad)

Aquí se muestra un ejemplo para una convolucón de una suma de 2 (X+Y) y 3 variables aleatorias (X+Y+Z)

In [44]:
# Realizar las convoluciones
f2 = convolusion(f_x, f_y, np.arange(11), np.arange(11))
print(f2)
print("Suma de f2:", np.sum(f2))

f3 = convolusion(f_x, f2, np.arange(11), np.arange(21))
print("Suma de f3:", np.sum(f3))

[1.83156389e-02 7.32625556e-02 1.46525111e-01 1.95366815e-01
 1.95366815e-01 1.56293452e-01 1.04195635e-01 5.95403626e-02
 2.97701813e-02 1.32311917e-02 5.29472547e-03 1.92715513e-03
 6.41937808e-04 1.95953268e-04 5.46602326e-05 1.38571219e-05
 3.16969133e-06 6.47790373e-07 1.16370572e-07 1.77575454e-08
 2.16207098e-09]
Suma de f2: 1.0000000000000022
Suma de f3: 1.000000000000003


Para hacer una suma de variables más grande lo que se hace es ir replicando la misma función que se creó arriba, solo que ahora se van reescribiendo los argumentos con los resultados que se obtienen en cada simulación

In [66]:
# MÁS CONVOLUCIONES
n = 50
for m in range(2, n + 1):
    if m == 2:
        suma = convolusion(f_x, f_y, np.arange(11), np.arange(11))
    else:
        suma = convolusion(f_x, suma, np.arange(11), np.arange(10 * (m - 1) + 1))

print("Resultado final de la convolución:", suma)
print("Suma de la convolución final:", np.sum(suma))

Resultado final de la convolución: [3.72007598e-044 3.72007598e-042 1.86003799e-040 6.20012663e-039
 1.55003166e-037 3.10006331e-036 5.16677219e-035 7.38110313e-034
 9.22637891e-033 1.02515321e-031 1.02515321e-030 9.31957466e-030
 7.76631221e-029 5.97408632e-028 4.26720451e-027 2.84480301e-026
 1.77800188e-025 1.04588346e-024 5.81046366e-024 3.05813877e-023
 1.52906938e-022 7.28128278e-022 3.30967399e-021 1.43898869e-020
 5.99578622e-020 2.39831449e-019 9.22428649e-019 3.41640241e-018
 1.22014372e-017 4.20739213e-017 1.40246404e-016 4.52407756e-016
 1.41377424e-015 4.28416437e-015 1.26004835e-014 3.60013814e-014
 1.00003838e-013 2.70280643e-013 7.11264855e-013 1.82375605e-012
 4.55939016e-012 1.11204639e-011 2.64772954e-011 6.15751066e-011
 1.39943427e-010 3.10985400e-010 6.76055237e-010 1.43841545e-009
 2.99669896e-009 6.11571246e-009 1.22314256e-008 2.39831890e-008
 4.61215208e-008 8.70217449e-008 1.61151396e-007 2.93002572e-007
 5.23218948e-007 9.17928117e-007 1.58263496e-006 2.6824

Al final se calcula la función de distribución

In [67]:
# FUNCIÓN DE DISTRIBUCIÓN
df = pd.DataFrame({'Rango': np.arange(10 * n + 1), 'Densidad': suma})
df['Distribucion'] = np.nan

# Cambiar el bucle para utilizar .iloc[] para asignar correctamente
for k in range(len(df)):
    df.iloc[k, df.columns.get_loc('Distribucion')] = np.sum(df['Densidad'][:k + 1])

print(df)

     Rango       Densidad  Distribucion
0        0   3.720076e-44  3.720076e-44
1        1   3.720076e-42  3.757277e-42
2        2   1.860038e-40  1.897611e-40
3        3   6.200127e-39  6.389888e-39
4        4   1.550032e-37  1.613931e-37
..     ...            ...           ...
496    496  2.000574e-209  1.000000e+00
497    497  3.641189e-211  1.000000e+00
498    498  5.079878e-213  1.000000e+00
499    499  4.832832e-215  1.000000e+00
500    500  2.353687e-217  1.000000e+00

[501 rows x 3 columns]
